In [33]:
import pandas as pd
from datetime import datetime, timedelta
N_DAYS_AGO=100

In [238]:
#If Df doesn't exist
sent_tweets_db = pd.DataFrame(columns=['name','cusip','twitter','tweet_date','tweet_text'])
sent_tweets_db.tail()
sent_tweets_fn = 'sent_tweets_db.csv'
sent_tweets_db.to_csv(sent_tweets_fn, index=False)

In [231]:
fn_puts = 'puts_by_stock.csv'

df_puts = pd.read_csv(fn_puts)


fn = 'df_company_info2_backup.csv'
df_tweet_handles = pd.read_csv(fn)

#df_to_tweet = df_puts.merge(df_twit_handles, how='inner',
#                             on = 'cusip')
#df_final = df_to_tweet.explode('fund_name').tail()

fn = 'sent_tweets_db.csv'
df_tweet_db = pd.read_csv(fn, dtype=str)
df_tweet_db['tweet_date'] = pd.to_datetime(df_tweet_db['tweet_date'])
df_tweet_db.tail()

,name,cusip,twitter,tweet_date,tweet_text
0,INVESCO QQQ TR,46090E103,InvescoUS,2024-03-04 22:28:03.185313,Hey @InvescoUS Did you know that on 2023-12-31...


In [232]:
#1 Create tweetable list
df_all_twitter_stats = df_puts.merge(df_tweet_handles, how='inner',
                                    left_on = ['nameOfIssuer', 'cusip'], right_on=['name','cusip'])
df_all_twitter_stats = df_all_twitter_stats.merge(df_tweet_db, how='left',
                                    on = ['name','cusip','twitter'])
df_all_twitter_stats_g = df_all_twitter_stats.groupby(['nameOfIssuer','cusip','twitter'],as_index=False).agg({'tweet_date':'max'}).rename(columns={'tweet_date':'max_tweet_date'})
#df_all_twitter_stats_g.tail()

today = datetime.today() 
n_days_ago = today - timedelta(days=N_DAYS_AGO)
#print(today)
print(n_days_ago)
df_tweet_eligible = df_all_twitter_stats_g[~(df_all_twitter_stats_g.max_tweet_date > n_days_ago)].reset_index(drop=True)
df_tweet_eligible.tail()
#df_tweetabl = df_all_twitter_stats_g[df_all_twitter_stats_g.max_tweet_date > 
#df_all_to_twee = df_all_twitter_stats[df_all_twitter_stats.]

2023-11-25 22:52:46.244094


,nameOfIssuer,cusip,twitter,max_tweet_date
459,WORKDAY INC,98138H101,workdaymagazine,NaT
460,WYNN RESORTS LTD,983134107,WynnLasVegas,NaT
461,ZILLOW GROUP INC,98954M101,ZillowGroup,NaT
462,ZILLOW GROUP INC,98954M200,ZillowGroup,NaT
463,ZSCALER INC,98980G102,ZscalerInc,NaT


In [233]:
df_puts['fund_name'] = df_puts['fund_name'].str.strip('[]').str.split(',')

df_puts_explode = df_puts.explode('fund_name')
df_puts_explode['fund_name'] = df_puts_explode['fund_name'].str.strip("'")
df_puts_explode.tail()

,nameOfIssuer,nameOfIssuer_link,cusip,value,fund_name
4104,VBI VACCINES INC CDA,"<a href=""http://www.google.com/search?q=stock ...",91822J202,59,CITADEL ADVISORS LLC
4105,EMCORE CORP,"<a href=""http://www.google.com/search?q=stock ...",290846203,49,CITADEL ADVISORS LLC
4106,EIGER BIOPHARMACEUTICALS INC,"<a href=""http://www.google.com/search?q=stock ...",28249U105,45,CITADEL ADVISORS LLC
4107,AETERNA ZENTARIS INC,"<a href=""http://www.google.com/search?q=stock ...",007975501,15,CITADEL ADVISORS LLC
4108,IBIO INC,"<a href=""http://www.google.com/search?q=stock ...",451033708,14,CITADEL ADVISORS LLC


In [234]:
fn = 'data_by_stocks.csv'
df_data_by_stocks = pd.read_csv(fn)
df_data_by_stocks_puts = df_data_by_stocks[df_data_by_stocks.putCall == 'Put']

df_final = df_tweet_eligible.merge(df_puts_explode, how = 'inner',
        left_on = ['nameOfIssuer','cusip'],
        right_on = ['nameOfIssuer','cusip']).merge(df_data_by_stocks_puts,
        left_on = ['nameOfIssuer','cusip','fund_name'],
        right_on = ['nameOfIssuer','cusip','fund_name']
        ).drop_duplicates().reset_index(drop=True)
df_final.sort_values('value_x',ascending=False).head().iloc[1]

twitter_name = df_final.sort_values('value_x',ascending=False).head().iloc[0].twitter
company_name = df_final[df_final.twitter == twitter_name].nameOfIssuer
company_cusip = df_final[df_final.twitter == twitter_name].cusip

print(twitter_name)
report_date = df_final[df_final.twitter == twitter_name].reportDate.max()


hf_link_dict = {}
for index, row in df_final[df_final.twitter == twitter_name].iterrows():
    #print(row)
    hf_link_dict[row.fund_name] = row.filingLink
print(hf_link_dict)


NVIDIAGeForce
{'CAPITAL FUND MANAGEMENT': 'https://www.sec.gov/Archives/edgar/data/1323645/0001323645-23-000005.txt'}


In [235]:
tweet = '''Hey @{} Did you know that on {} the following hedge funds reported a short against you? Check these link(s): 

{}'''.format(twitter_name,report_date, hf_link_dict).replace("',","\n")

print(tweet)

Hey @NVIDIAGeForce Did you know that on 2023-12-31 the following hedge funds reported a short against you? Check these link(s): 

{'CAPITAL FUND MANAGEMENT': 'https://www.sec.gov/Archives/edgar/data/1323645/0001323645-23-000005.txt'}


In [237]:
fn = 'sent_tweets_db.csv'
df_tweet_db = pd.read_csv(fn, dtype=str)
df_tweet_db['cusip']=  df_tweet_db['cusip'].astype(str)  
data_to_add = {
    'name':company_name, 
    'cusip':company_cusip, 
    'twitter':twitter_name, 
    'tweet_date':today, 
    'tweet_text':tweet}
df_to_add = pd.DataFrame(data_to_add).drop_duplicates().reset_index(drop=True)
df_tweet_db =pd.concat([df_tweet_db, df_to_add]).drop_duplicates().reset_index(drop=True)

fn = 'sent_tweets_db.csv'
df_tweet_db.to_csv(fn, index=False)
df_tweet_db.tail()

,name,cusip,twitter,tweet_date,tweet_text
0,INVESCO QQQ TR,46090E103,InvescoUS,2024-03-04 22:28:03.185313,Hey @InvescoUS Did you know that on 2023-12-31...
1,NVIDIA CORPORATION,67066G104,NVIDIAGeForce,2024-03-04 22:52:46.244094,Hey @NVIDIAGeForce Did you know that on 2023-1...
2,NVIDIA CORPORATION,67066G104,NVIDIAGeForce,2024-03-04 22:52:46.244094,Hey @NVIDIAGeForce Did you know that on 2023-1...


In [227]:
#If Df doesn't exist
sent_tweets_db = pd.DataFrame(columns=['name','cusip','twitter','tweet_date','tweet_text'])
sent_tweets_db.tail()
sent_tweets_fn = 'sent_tweets_db.csv'
sent_tweets_db.to_csv(sent_tweets_fn, index=False)

In [193]:
fn = 'sent_tweets_db.csv'
df_tweet_db = pd.read_csv(fn)
df_tweet_db.info

<bound method DataFrame.info of              name          cusip    twitter                  tweet_date  \
0  INVESCO QQQ TR  4.609000e+107  InvescoUS  2024-03-04 22:13:08.567871   

                                          tweet_text  
0  Hey @InvescoUS Did you know that on 2023-12-31...  >